In [2]:
import math
import numpy
from matplotlib import pyplot
%matplotlib inline

In [3]:
# Set the front family and size to use for Matplotlib figures.
pyplot.rcParams['font.family']='serif'
pyplot.rcParams['font.size']=16

In [4]:
# Set parameters.
g = 9.81 # gravitational accelertion
vt = 4.9 # trim velocity (m/s)
CD = 1.0/5.0 # dragcoefficient
CL = 1.0 # lift coefficient

# Set the initial conditions.
v0 = 6.5 # start at the trim velocity
theta0 = -0.1 # trajectory angle
x0 = 0.0 # horizontal position
y0 = 2.0 # vertical position (altitude)

In [5]:
def rhs_phugoid(u, CL, CD, g, vt):
    """
    Returns the right-hand side of the phugoid system of equations.
    
    Parameters
    ----------
    u : list or numpy.ndarray
        Solution at the previous time step
        as alist or 1d array of four floats.
    CL : float
        Lift coefficient
    CD : float
        Drag coefficient
    g : float
        Gravitational acceleration.
    vt : float
        Trim velocity.
        
    Returns
    -------
    rhs : numpy.ndarray
        The right-hand side of the system
        as a 1D array of four floats.
    """
    v, theta, x, y = u
    rhs = numpy.array([-g*math.sin(theta)-CD/CL*g/vt**2*v**2,
                      -g*math.cos(theta)/v+g/vt**2*v,
                      v*math.cos(theta),
                      v*math.sin(theta)])
    return rhs

def euler_step(u, f, dt, *args):
    """
    Returns the solution at the next time step using Euler's method.
    
    Parameters
    -----------
    u : numpy.ndarray
        Solution at the previous time step
        as a 1D array of floats.
    f : function
        Function to compute the right-hand side of the system.
    dt : float
        Time-step size.
    args : tuple, optional
        Positional arguments to pass to the function f.
        
    Returns
    --------
    u_new : numpy.ndarray
        The solution at the next time step
        as a 1D array of floats.
    """
    u_new = u + dt*f(u, *args)
    return u_new

def l1_diff(u_coarse, u_fine, dt):
    """
    Returns the difference in the L1-norm between the solution on 
    a coarse grid and the solution on a fine grid.
    
    Parameters
    ----------
    u_coarse : numpy.ndarray
        Solution on the coarse grid as a 1D array of floats.
    u_fine : numpy.ndarray
        Solution on the fine grid as a 1D array of floats.
    dt : float
        Time-step size.
        
    Returns
    --------
    diff : float
        The difference between the two solutions in the L1-norm
        scaled by the time-step size.
    """
    N_coarse = u_coarse.shape[0]
    N_fine = u_fine.shape[0]
    ratio = math.ceil(N_fine/N_coarse)
    diff = dt*numpy.sum(numpy.abs(u_coarse-u_fine[::ratio]))
    return diff

In [6]:
def rk2_step(u, f, dt, *args):
    """
    Returns the solution at the next time step using 2nd-order
    Runge-Kutta method.
    
    Prameters
    ---------
    u : numpy.ndarray
        Solution at the previous time step
        as a 1D array of floats.
    f : function
        Function to compute the right-hand side of the system.
    dt : float
        Time-step size.
    args : tuple, optional
        Positional arguments to pass to the function f.
        
    Returns
    -------
    u_new : numpy.ndarray
        The solution at the next time step
        as a 1D array of floats.
    """
    u_star=u+0.5*dt*f(u,*args)
    u_new = u+dt*f(u_star, *args)
    return u_new

In [7]:
T=15.0 # Length of time interval
dt=0.01 #time-step size
N=int(T/dt)+1 # number of time steps

#Create arrays to store the solution at each time-step.
u_euler=numpy.empty((N,4))
u_rk2=numpy.empty((N,4))

# Set the initial conditions.
u_euler[0] = numpy.array([v0, theta0, x0, y0])
u_rk2[0]=numpy.array([v0, theta0, x0, y0])

# Time integration with both method.
for n in range (N-1):
    u_euler[n+1]=euler_step(u_euler[n], rhs_phugoid, dt, 
                           CL, CD, g, vt)
    u_rk2[n+1]=rk2_step(u_rk2[n], rhs_phugoid, dt,
                       CL, CD, g, vt)

In [8]:
# Get the glider's position over the time.
x_euler=u_euler[:,2]
y_euler=u_euler[:3]
x_rk2=u_rk2[:,2]
y_rk2=u_rk2[:,3]

In [ ]:
# Get the index of the first negative element of y_euler.
idx_negative_euler=numpy.where(y_euler<0.0)[0]
if len(idx_negative_euler)0==0:
    idx_ground_euler = N-1
    print('[Euler] Glider has not touched ground yet!')
else:
    idx_ground_euler=idx_negative_euler[0]
#Get the index of the first negatie element of y_rk2.
idx_negative_rk2=numpy.where(y_rk2<0.0)[0]
if len(idx_negative_rk2)==0:
    idx_ground_rk2=N-1
    print('[RK2] Glider has not touched ground yet!')
else:
    idx_ground_rk2=idk_